In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import csv

In [2]:
def process_raw_data(city,filename):
    
    data = pd.read_csv("data/" + filename, header=None)
    print("filename", filename)
    print("data", data.head(1))
    print()
    
    for i in data.iterrows():
        resume_id = i[1][0]
        info = i[1][1]

        resume_soup = BeautifulSoup(info, "html.parser")
        containers = resume_soup.findAll(name="div", attrs={"class":"items-container"})

        for container in containers:
            container_type = container['id']

            if container_type == 'work-experience-items':
                all_work = resume_soup.findAll(name="div", attrs={"id":re.compile('workExperience.*')})

                for work_pos in all_work:

                    work_title = work_pos.find(name="p", attrs={"class":"work_title title"})
                    if work_title: work_title = work_title.next
                    else: work_title = ''

                    work_company = work_pos.find(name="div", attrs={"class":"work_company"})
                    if work_company:work_company = work_company.next.next
                    else: work_company = ''

                    addressLocality = work_pos.find(name="span", attrs={"itemprop":"addressLocality"})
                    if addressLocality: addressLocality = addressLocality.next
                    else: addressLocality = ''

                    work_dates = work_pos.find(name="p", attrs={"class":"work_dates"})
                    if work_dates: work_dates = work_dates.next
                    else: work_dates = ''

                    work_descript = work_pos.find(name="p", attrs={"class":"work_description"})
                    if work_descript: work_descript = re.sub('<[^<]+?>', '.', str(work_descript))
                    else: work_descript = ''

                    row = [city, resume_id, container_type, work_title, work_company, addressLocality, work_dates, work_descript]
                    with open('00_resumes_work.csv', 'a') as myfile:
                        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                        wr.writerow(row)                

            if container_type == 'education-items':
                all_edu = resume_soup.findAll(name="div", attrs={"class":re.compile("education-section*")})
                for edu in all_edu:
                    edu_title = edu.find(name="p", attrs={"class":"edu_title"})
                    if edu_title: edu_title = edu_title.next
                    else: edu_title = ''

                    edu_school = edu.find(name="div", attrs={"class":"edu_school"})
                    if edu_school: edu_school = edu_school.next.next
                    else: edu_school = ''

                    edu_dates = edu.find(name="p", attrs={"class":"edu_dates"})
                    if edu_dates: edu_dates = edu_dates.next
                    else: edu_dates = ''

                    edu_addressLocality = edu.find(name="span", attrs={"itemprop":"addressLocality"})
                    if edu_addressLocality: edu_addressLocality = edu_addressLocality.next
                    else: edu_addressLocality = ''

                    row = [city, resume_id, container_type, edu_title, edu_school, edu_addressLocality, edu_dates]
                    with open('00_resumes_education.csv', 'a') as myfile:
                        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                        wr.writerow(row)                

            if container_type == 'skills-items':
                all_skills = resume_soup.findAll(name="span", attrs={"class":"skill-text"})
                for skill in all_skills:
                    if skill: skill = skill.next
                    else: skill = ''

                    row = [city, resume_id, container_type, skill]
                    with open('00_resumes_skill.csv', 'a') as myfile:
                        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                        wr.writerow(row)                

            if container_type == 'additionalinfo-items':
                add_info = resume_soup.find(name="div", attrs={"id":"additionalinfo-section"})
                if add_info: add_info = re.sub('<[^<]+?>', '.', str(add_info))
                else: add_info = ''

                row = [city, resume_id, container_type, add_info]
                with open('00_resumes_addinfo.csv', 'a') as myfile:
                    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                    wr.writerow(row)                

            if container_type in ['certification-items','group-items','award-items','patent-items','publication-items','link-items']:
                data_type = container_type.partition('-')[0]
                all_groupings = resume_soup.findAll(name="div", attrs={"class":re.compile(data_type + "-section*")})
                for grouping in all_groupings:

                    title = grouping.find(name="p", attrs={"class": data_type + "_title"})
                    if title: title = title.next
                    else: title = ''

                    date = grouping.find(name="p", attrs={"class": data_type + "_date"})
                    if date: date = date.next
                    else: date = ''

                    url = grouping.find(name="p", attrs={"class": data_type + "_url"})
                    if url: url = url.next.next
                    else: url = ''

                    descript = grouping.find(name="p", attrs={"class": data_type + "_description"})
                    if descript: 
                        descript = re.sub('<[^<]+?>', '.', str(descript))
                    else: descript = ''

                    row = [city, resume_id, container_type, title, date, descript, url]
                    with open('00_resumes_cert.csv', 'a') as myfile:
                        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                        wr.writerow(row)

            if container_type == 'military-items':
                all_military = resume_soup.findAll(name="div", attrs={"class":re.compile("military-section*")})
                for military in all_military:

                    military_country = military.find(name="p", attrs={"military_country"})
                    if military_country: 
                        military_country = re.sub('<[^<]+?>', '.', str(military_country))
                    else: military_country = ''

                    military_rank = military.find(name="p", attrs={"military_rank"})
                    if military_rank: 
                        military_rank = re.sub('<[^<]+?>', '.', str(military_rank))
                    else: military_rank = ''

                    military_branch = military.find(name="p", attrs={"military_branch"})
                    if military_branch: 
                        military_branch = re.sub('<[^<]+?>', '.', str(military_branch))
                    else: military_branch = ''

                    military_date = military.find(name="p", attrs={"military_date"})
                    if military_date: military_date = military_date.next
                    else: military_date = ''

                    military_description = military.find(name="p", attrs={"military_description"})
                    if military_description: 
                        military_description = re.sub('<[^<]+?>', '.', str(military_description))
                    else: military_description = ''

                    military_commendations_title = military.find(name="p", attrs={"military_commendations_title"})
                    if military_commendations_title: 
                        military_commendations_title = re.sub('<[^<]+?>', '.', str(military_commendations_title))
                    else: military_commendations_title = ''
                        
                    row = [city, resume_id, container_type, military_country, military_rank, military_branch, \
                           military_date, military_description, military_commendations_title]
                    with open('00_resumes_military.csv', 'a') as myfile:
                        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                        wr.writerow(row)

In [3]:
files = [
    'atlanta_resume_records.csv'
#     'boston_resume_records.csv'
#     'sf_resume_records.csv'
#     'austin_resume_records.csv',
#     'dallas_resume_records.csv',
#     'dc_resume_records.csv',
#     'denver_resume_records.csv',
#     'detroit_resume_records.csv',
#     'minneapolis_resume_records.csv',
#     'philly_resume_records.csv',
#     'raleigh_resume_records.csv',
#     'sanjose_resume_records.csv',
#     'seattle_resume_records.csv',
#     'atlanta_resume_data_01.csv',
#     'atlanta_resume_data_02.csv',
#     'atlanta_resume_data_03.csv',
#     'atlanta_resume_data_04.csv',
#     'atlanta_resume_data_05.csv',
#     'austin_resume_data_01.csv',
#     'austin_resume_data_02.csv',
#     'austin_resume_data_03.csv',
#     'boston_resume_data_01.csv',
#     'boston_resume_data_02.csv',
#     'boston_resume_data_03.csv',
#     'boston_resume_data_04.csv',
#     'dc_resume_data_01.csv',
#     'dc_resume_data_02.csv',
#     'dc_resume_data_03.csv',
#     'dc_resume_data_04.csv',
#     'denver_resume_data_01.csv',
#     'denver_resume_data_02.csv',
#     'denver_resume_data_03.csv',
#     'denver_resume_data_04.csv',
#     'miami_resume_data_01.csv',
#     'miami_resume_data_02.csv',
#     'miami_resume_data_03.csv',
#     'miami_resume_data_04.csv',
#     'minn_resume_data_01.csv',
#     'minn_resume_data_02.csv',
#     'minn_resume_data_03.csv',
#     'minn_resume_data_04.csv',
#     'minn_resume_data_05.csv',
#     'minn_resume_data_06.csv',
#     'minn_resume_data_07.csv',
#     'nyc_resume_data_01.csv',
#     'nyc_resume_data_02.csv',
#     'nyc_resume_data_03.csv',
#     'nyc_resume_data_04.csv',
#     'nyc_resume_data_05.csv',
#     'nyc_resume_data_06.csv',
#     'nyc_resume_data_07.csv',
#     'nyc_resume_data_08.csv',
#     'nyc_resume_data_09.csv',
#     'seattle_resume_data_01.csv',
#     'seattle_resume_data_02.csv',
#     'seattle_resume_data_03.csv',
#     'sf_resume_data_01.csv',
#     'sf_resume_data_02.csv',
#     'sf_resume_data_03.csv',
]

In [4]:
for filename in files:
    city = filename.split("_")[0]
    process_raw_data(city,filename)

filename atlanta_resume_records.csv
data                   0                                                  1
0  0002bfaa5b887f08  <div class="data_display" id="basic_info_cell"...



In [82]:
directory = '/Users/kwheatley/Desktop/Capstone/gcloud_data/'

resumes_addinfo = pd.read_csv(directory+"00_resumes_addinfo.csv",header=None)
resumes_cert = pd.read_csv(directory+"00_resumes_cert.csv",header=None)
resumes_education = pd.read_csv(directory+"00_resumes_education.csv",header=None)
resumes_military = pd.read_csv(directory+"00_resumes_military.csv",header=None)
resumes_skill = pd.read_csv(directory+"00_resumes_skill.csv",header=None)
resumes_work = pd.read_csv(directory+"00_resumes_work.csv",header=None)

In [83]:
resumes_addinfo.columns = ['search_city', 'resume_id', 'container_type', 'skill']
print(resumes_addinfo.count())
del resumes_addinfo['search_city']
resumes_addinfo_dd = resumes_addinfo.drop_duplicates()
print(resumes_addinfo_dd.count())

search_city       426239
resume_id         426239
container_type    426239
skill             426239
dtype: int64
resume_id         391637
container_type    391637
skill             391637
dtype: int64


In [84]:
resumes_cert.columns = ['search_city', 'resume_id', 'container_type', 'title', 'date', 
                'descript', 'url']
print(resumes_cert.count())
del resumes_cert['search_city']
resumes_cert_dd = resumes_cert.drop_duplicates()
print(resumes_cert_dd.count())

search_city       498538
resume_id         498538
container_type    498538
title             351046
date              242654
descript          142672
url               166887
dtype: int64
resume_id         448287
container_type    448287
title             317234
date              219685
descript          129113
url               147975
dtype: int64


In [85]:
resumes_education.columns = ['search_city', 'resume_id', 'container_type', 'edu_title', 
                             'edu_school', 'edu_location', 'edu_dates']
print(resumes_education.count())
del resumes_education['search_city']
resumes_education_dd = resumes_education.drop_duplicates()
print(resumes_education_dd.count())

search_city       1206196
resume_id         1206196
container_type    1206196
edu_title         1098727
edu_school        1175219
edu_location       769334
edu_dates          838240
dtype: int64
resume_id         1099394
container_type    1099394
edu_title          999545
edu_school        1071508
edu_location       702466
edu_dates          767455
dtype: int64


In [86]:
resumes_military.columns = ['search_city', 'resume_id', 'container_type', 'military_country',
    'military_rank', 'military_branch', 'military_date', 'military_description', 'military_commendations_title']
print(resumes_military.count())
del resumes_military['search_city']
resumes_military_dd = resumes_military.drop_duplicates()
print(resumes_military_dd.count())

search_city                     15612
resume_id                       15612
container_type                  15612
military_country                15612
military_rank                   15612
military_branch                 15612
military_date                   13418
military_description             8918
military_commendations_title     5507
dtype: int64
resume_id                       14258
container_type                  14258
military_country                14258
military_rank                   14258
military_branch                 14258
military_date                   12252
military_description             8155
military_commendations_title     5018
dtype: int64


In [87]:
resumes_skill.columns = ['search_city', 'resume_id', 'container_type', 'skill']
print(resumes_skill.count())
del resumes_skill['search_city']
resumes_skill_dd = resumes_skill.drop_duplicates()
print(resumes_skill_dd.count())

search_city       2382369
resume_id         2382369
container_type    2382369
skill             2382361
dtype: int64
resume_id         2138983
container_type    2138983
skill             2138976
dtype: int64


In [88]:
resumes_work.columns = ['search_city', 'resume_id', 'container_type', 'work_title', 
                        'work_company','work_location', 'work_dates', 'work_descript']
print(resumes_work.count())
del resumes_work['search_city']
resumes_work_dd = resumes_work.drop_duplicates()
print(resumes_work_dd.count())

search_city       3827215
resume_id         3827215
container_type    3827215
work_title        3767137
work_company      3744715
work_location      434533
work_dates        3801669
work_descript     3581068
dtype: int64
resume_id         3503769
container_type    3503769
work_title        3448142
work_company      3428044
work_location      401983
work_dates        3479698
work_descript     3276632
dtype: int64


In [95]:
resumes_addinfo_dd.to_csv(directory+"00_resumes_addinfo_dd.csv",index=False)
resumes_cert_dd.to_csv(directory+"00_resumes_cert_dd.csv",index=False)
resumes_education_dd.to_csv(directory+"00_resumes_education_dd.csv",index=False)
resumes_military_dd.to_csv(directory+"00_resumes_military_dd.csv",index=False)
resumes_skill_dd.to_csv(directory+"00_resumes_skill_dd.csv",index=False)
resumes_work_dd.to_csv(directory+"00_resumes_work_dd.csv",index=False)

# End